# Eat Safe, Love

## Part 1: Database and Jupyter Notebook Set Up

Import the data provided in the `establishments.json` file from your Terminal. Name the database `uk_food` and the collection `establishments`.

Within this markdown cell, copy the line of text you used to import the data from your Terminal. This way, future analysts will be able to repeat your process.

e.g.: Import the dataset with mongoimport --type json -d met -c artifacts --drop --jsonArray 

In [1]:
# Import dependencies
#deal with Mongo
from pymongo import MongoClient

#help with json
import json
from pprint import pprint

#deal with api requests
import requests 
import time

#data science
import pandas as pd
import numpy as np

#data viz
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)

In [3]:
# confirm that our new database was created
print(mongo.list_database_names())

['admin', 'autosaurus', 'classDB', 'config', 'epa', 'fruit_DB', 'local', 'met', 'petsitly_marketing', 'travel_db', 'uk_food']


In [4]:
# assign the uk_food database to a variable name
db = mongo['uk_food']

In [6]:
# review the collections in our new database
print(db.list_collection_names())

['establishments']


In [10]:
# review a document in the establishments collection
db.establishments.find_one()

{'_id': ObjectId('668f46dfabf908d119d1181d'),
 'FHRSID': 254719,
 'ChangesByServerID': 0,
 'LocalAuthorityBusinessID': 'PI/000069980',
 'BusinessName': 'Refreshment Kiosk',
 'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': 1,
 'AddressLine1': 'The Bay',
 'AddressLine2': 'St Margarets Bay',
 'AddressLine3': 'Kent',
 'AddressLine4': '',
 'PostCode': 'CT15 6DY',
 'Phone': '',
 'RatingValue': '5',
 'RatingKey': 'fhrs_5_en-gb',
 'RatingDate': '2022-03-24T00:00:00',
 'LocalAuthorityCode': '182',
 'LocalAuthorityName': 'Dover',
 'LocalAuthorityWebSite': 'http://www.dover.gov.uk/',
 'LocalAuthorityEmailAddress': 'publicprotection@dover.gov.uk',
 'scores': {'Hygiene': 0, 'Structural': 5, 'ConfidenceInManagement': 5},
 'SchemeType': 'FHRS',
 'geocode': {'longitude': '1.387974', 'latitude': '51.152225'},
 'RightToReply': '',
 'Distance': 4587.347174863443,
 'NewRatingPending': False,
 'meta': {'dataSource': None,
  'extractDate': '0001-01-01T00:00:00',
  'itemCount': 0,
  'returncode

In [11]:
# assign the collection to a variable
establishments = db['establishments']

## Part 2: Update the Database

1. An exciting new halal restaurant just opened in Greenwich, but hasn't been rated yet. The magazine has asked you to include it in your analysis. Add the following restaurant "Penang Flavours" to the database.

In [17]:
# Create a dictionary for the new restaurant data
data = [{
    "BusinessName":"Penang Flavours",
    "BusinessType":"Restaurant/Cafe/Canteen",
    "BusinessTypeID":"",
    "AddressLine1":"Penang Flavours",
    "AddressLine2":"146A Plumstead Rd",
    "AddressLine3":"London",
    "AddressLine4":"",
    "PostCode":"SE18 7DY",
    "Phone":"",
    "LocalAuthorityCode":"511",
    "LocalAuthorityName":"Greenwich",
    "LocalAuthorityWebSite":"http://www.royalgreenwich.gov.uk",
    "LocalAuthorityEmailAddress":"health@royalgreenwich.gov.uk",
    "scores":{
        "Hygiene":"",
        "Structural":"",
        "ConfidenceInManagement":""
    },
    "SchemeType":"FHRS",
    "geocode":{
        "longitude":"0.08384000",
        "latitude":"51.49014200"
    },
    "RightToReply":"",
    "Distance":4623.9723280747176,
    "NewRatingPending":True
}]

In [18]:
# Insert the new restaurant into the collection
db.establishments.insert_many(data)

InsertManyResult([ObjectId('668f4a3cec9c0a8f27dd8ce1')], acknowledged=True)

In [22]:
# Check that the new restaurant was inserted
query = {
    'BusinessName': 'Penang Flavours'
}
results = establishments.find(query)

df = pd.DataFrame(results)
df

,_id,BusinessName,BusinessType,BusinessTypeID,AddressLine1,AddressLine2,AddressLine3,AddressLine4,PostCode,Phone,LocalAuthorityCode,LocalAuthorityName,LocalAuthorityWebSite,LocalAuthorityEmailAddress,scores,SchemeType,geocode,RightToReply,Distance,NewRatingPending
0,668f4a3cec9c0a8f27dd8ce1,Penang Flavours,Restaurant/Cafe/Canteen,,Penang Flavours,146A Plumstead Rd,London,,SE18 7DY,,511,Greenwich,http://www.royalgreenwich.gov.uk,health@royalgreenwich.gov.uk,"{'Hygiene': '', 'Structural': '', 'ConfidenceI...",FHRS,"{'longitude': '0.08384000', 'latitude': '51.49...",,4623.972328,True


2. Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the `BusinessTypeID` and `BusinessType` fields.

In [25]:
# Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the BusinessTypeID and BusinessType fields
query = {
    'BusinessType': 'Restaurant/Cafe/Canteen'
}
fields = {'BusinessTypeID':1, 'BusinessType':1}
results1 = establishments.find(query, fields)

df1 = pd.DataFrame(results1)
df1

,_id,BusinessType,BusinessTypeID
0,668f46dfabf908d119d1181d,Restaurant/Cafe/Canteen,1
1,668f46dfabf908d119d11820,Restaurant/Cafe/Canteen,1
2,668f46dfabf908d119d11822,Restaurant/Cafe/Canteen,1
3,668f46dfabf908d119d11823,Restaurant/Cafe/Canteen,1
4,668f46dfabf908d119d11824,Restaurant/Cafe/Canteen,1
...,...,...,...
10091,668f46e4abf908d119d1b36a,Restaurant/Cafe/Canteen,1
10092,668f46e4abf908d119d1b36f,Restaurant/Cafe/Canteen,1
10093,668f46e4abf908d119d1b370,Restaurant/Cafe/Canteen,1
10094,668f46e4abf908d119d1b37e,Restaurant/Cafe/Canteen,1


3. Update the new restaurant with the `BusinessTypeID` you found.

In [26]:
# Update the new restaurant with the correct BusinessTypeID
db.establishments.update_one({'BusinessName': 'Penang Flavours'}, {'$set': {'BusinessTypeID': 1}})

UpdateResult({'n': 1, 'nModified': 1, 'ok': 1.0, 'updatedExisting': True}, acknowledged=True)

In [27]:
# Confirm that the new restaurant was updated
query = {
    'BusinessName': 'Penang Flavours'
}
results = establishments.find(query)

df = pd.DataFrame(results)
df

,_id,BusinessName,BusinessType,BusinessTypeID,AddressLine1,AddressLine2,AddressLine3,AddressLine4,PostCode,Phone,LocalAuthorityCode,LocalAuthorityName,LocalAuthorityWebSite,LocalAuthorityEmailAddress,scores,SchemeType,geocode,RightToReply,Distance,NewRatingPending
0,668f4a3cec9c0a8f27dd8ce1,Penang Flavours,Restaurant/Cafe/Canteen,1,Penang Flavours,146A Plumstead Rd,London,,SE18 7DY,,511,Greenwich,http://www.royalgreenwich.gov.uk,health@royalgreenwich.gov.uk,"{'Hygiene': '', 'Structural': '', 'ConfidenceI...",FHRS,"{'longitude': '0.08384000', 'latitude': '51.49...",,4623.972328,True


4. The magazine is not interested in any establishments in Dover, so check how many documents contain the Dover Local Authority. Then, remove any establishments within the Dover Local Authority from the database, and check the number of documents to ensure they were deleted.

994

In [36]:
# Find how many documents have LocalAuthorityName as "Dover"
specific_id = 'Dover'

count = establishments.count_documents({'LocalAuthorityName': specific_id})
count

994

In [40]:
# Delete all documents where LocalAuthorityName is "Dover"
query = {'LocalAuthorityName': 'Dover'}
establishments.delete_many(query)

DeleteResult({'n': 994, 'ok': 1.0}, acknowledged=True)

In [41]:
# Check if any remaining documents include Dover
establishments.count_documents(query)

0

In [42]:
# Check that other documents remain with 'find_one'
establishments.find_one()

{'_id': ObjectId('668f46dfabf908d119d11b04'),
 'FHRSID': 1043695,
 'ChangesByServerID': 0,
 'LocalAuthorityBusinessID': 'PI/000073616',
 'BusinessName': 'The Pavilion',
 'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': 1,
 'AddressLine1': 'East Cliff Pavilion',
 'AddressLine2': 'Wear Bay Road',
 'AddressLine3': 'Folkestone',
 'AddressLine4': 'Kent',
 'PostCode': 'CT19 6BL',
 'Phone': '',
 'RatingValue': '5',
 'RatingKey': 'fhrs_5_en-gb',
 'RatingDate': '2018-04-04T00:00:00',
 'LocalAuthorityCode': '188',
 'LocalAuthorityName': 'Folkestone and Hythe',
 'LocalAuthorityWebSite': 'http://www.folkestone-hythe.gov.uk',
 'LocalAuthorityEmailAddress': 'foodteam@folkestone-hythe.gov.uk',
 'scores': {'Hygiene': 5, 'Structural': 5, 'ConfidenceInManagement': 5},
 'SchemeType': 'FHRS',
 'geocode': {'longitude': '1.195625', 'latitude': '51.083812'},
 'RightToReply': '',
 'Distance': 4591.765489457773,
 'NewRatingPending': False,
 'meta': {'dataSource': None,
  'extractDate': '0001-01-01

5. Some of the number values are stored as strings, when they should be stored as numbers.

Use `update_many` to convert `latitude` and `longitude` to decimal numbers.

In [43]:
# Change the data type from String to Decimal for longitude and latitude
establishments.update_many({}, [ {'$set':{ "geocode.longitude" : {'$toDecimal': "$geocode.longitude"}}} ])
establishments.update_many({}, [ {'$set':{ "geocode.latitude" : {'$toDecimal': "$geocode.latitude"}}} ])

UpdateResult({'n': 38786, 'nModified': 38786, 'ok': 1.0, 'updatedExisting': True}, acknowledged=True)

Use `update_many` to convert `RatingValue` to integer numbers.

In [44]:
# Set non 1-5 Rating Values to Null
non_ratings = ["AwaitingInspection", "Awaiting Inspection", "AwaitingPublication", "Pass", "Exempt"]
establishments.update_many({"RatingValue": {"$in": non_ratings}}, [ {'$set':{ "RatingValue" : None}} ])

UpdateResult({'n': 4091, 'nModified': 4091, 'ok': 1.0, 'updatedExisting': True}, acknowledged=True)

In [45]:
# Change the data type from String to Integer for RatingValue
establishments.update_many({}, [ {'$set':{ "RatingValue" : {'$toInt': "$RatingValue"}}} ])

UpdateResult({'n': 38786, 'nModified': 34695, 'ok': 1.0, 'updatedExisting': True}, acknowledged=True)

In [46]:
# Check that the coordinates and rating value are now numbers
establishments.find_one()

{'_id': ObjectId('668f46dfabf908d119d11b04'),
 'FHRSID': 1043695,
 'ChangesByServerID': 0,
 'LocalAuthorityBusinessID': 'PI/000073616',
 'BusinessName': 'The Pavilion',
 'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': 1,
 'AddressLine1': 'East Cliff Pavilion',
 'AddressLine2': 'Wear Bay Road',
 'AddressLine3': 'Folkestone',
 'AddressLine4': 'Kent',
 'PostCode': 'CT19 6BL',
 'Phone': '',
 'RatingValue': 5,
 'RatingKey': 'fhrs_5_en-gb',
 'RatingDate': '2018-04-04T00:00:00',
 'LocalAuthorityCode': '188',
 'LocalAuthorityName': 'Folkestone and Hythe',
 'LocalAuthorityWebSite': 'http://www.folkestone-hythe.gov.uk',
 'LocalAuthorityEmailAddress': 'foodteam@folkestone-hythe.gov.uk',
 'scores': {'Hygiene': 5, 'Structural': 5, 'ConfidenceInManagement': 5},
 'SchemeType': 'FHRS',
 'geocode': {'longitude': Decimal128('1.195625'),
  'latitude': Decimal128('51.083812')},
 'RightToReply': '',
 'Distance': 4591.765489457773,
 'NewRatingPending': False,
 'meta': {'dataSource': None,
  'e